# Model Deployment

## Load Trained Model Random Forest

In [11]:
import numpy as np
import pandas as pd
import joblib
import gradio as gr

In [5]:
model_path = '../model_config/best_model/rf_model_bundle.pkl'

bundle = joblib.load(model_path)

model = bundle['model']
threshold = bundle['threshold']

print(f'Model and threshold loaded successfully - Threshold: {threshold: .4f}')

Model and threshold loaded successfully - Threshold:  0.0200


In [9]:
preprocessor = model.named_steps["preprocessor"]

original_input_features = preprocessor.feature_names_in_

print(original_input_features)


['step' 'type' 'amount' 'oldbalanceOrig' 'newbalanceOrig'
 'balanceChangeOrig' 'OrigEmptyBefore' 'OrigEmptyAfter' 'oldbalanceDest'
 'newbalanceDest' 'balanceChangeDest' 'DestEmptyBefore' 'DestEmptyAfter'
 'nameDest_freq' 'nameDest_fraud_rate']


## Run sample prediction

In [12]:
X_sample = pd.DataFrame([{
    "step": 1,
    "amount": 5000.0,
    "oldbalanceOrig": 10000.0,
    "newbalanceOrig": 5000.0,
    "oldbalanceDest": 0.0,
    "newbalanceDest": 0.0,
    "balanceChangeOrig": 5000.0,
    "balanceChangeDest": 0.0,
    "nameDest_freq": 0.0,
    "nameDest_fraud_rate": 0.0,
    "type": "CASH_OUT",
    "OrigEmptyBefore": 0,
    "OrigEmptyAfter": 0,
    "DestEmptyBefore": 0,
    "DestEmptyAfter": 0
}])

proba = model.predict_proba(X_sample)[0][1]
pred = int(proba >= threshold)
label = "Fraud" if pred else "Not Fraud"

print(f"Prediction: {label} (Probability: {proba:.2f})")

Prediction: Not Fraud (Probability: 0.00)


## Gradio UI

In [13]:
def predict_fraud(step, amount, oldbalanceOrig, newbalanceOrig,
                  oldbalanceDest, newbalanceDest,
                  nameDest_freq, nameDest_fraud_rate,
                  type, OrigEmptyBefore, OrigEmptyAfter, DestEmptyBefore, DestEmptyAfter):
    balanceChangeOrig = oldbalanceOrig - newbalanceOrig
    balanceChangeDest = newbalanceDest - oldbalanceDest

    X = pd.DataFrame([{
        "step": step,
        "amount": amount,
        "oldbalanceOrig": oldbalanceOrig,
        "newbalanceOrig": newbalanceOrig,
        "oldbalanceDest": oldbalanceDest,
        "newbalanceDest": newbalanceDest,
        "balanceChangeOrig": balanceChangeOrig,
        "balanceChangeDest": balanceChangeDest,
        "nameDest_freq": nameDest_freq,
        "nameDest_fraud_rate": nameDest_fraud_rate,
        "type": type,
        "OrigEmptyBefore": OrigEmptyBefore,
        "OrigEmptyAfter": OrigEmptyAfter,
        "DestEmptyBefore": DestEmptyBefore,
        "DestEmptyAfter": DestEmptyAfter
    }])

    prob = model.predict_proba(X)[0][1]
    pred = int(prob >= threshold)
    return f"{'Fraud' if pred else 'Not Fraud'} (Probability: {prob:.2f})"


In [17]:
demo = gr.Interface(
    fn=predict_fraud,
    inputs=[
        gr.Number(label="Transaction Step", info="Time unit since system start"),
        gr.Number(label="Transaction Amount ($)", info="Total amount of the transaction"),
        gr.Number(label="Sender's Balance Before", info="Balance before transaction"),
        gr.Number(label="Sender's Balance After", info="Balance after transaction"),
        gr.Number(label="Recipient's Balance Before", info="Recipient balance before transaction"),
        gr.Number(label="Recipient's Balance After", info="Recipient balance after transaction"),
        gr.Number(label="Recipient Account Frequency", info="Number of prior transactions to recipient"),
        gr.Number(label="Recipient Fraud Rate", info="Historical fraud rate for recipient (0–1)"),
        gr.Radio(["CASH_OUT", "TRANSFER", "PAYMENT", "CASH_IN"], label="Transaction Type", info="CASH_OUT & TRANSFER are riskier"),
        gr.Radio([0, 1], label="Sender Balance Empty Before?", info="1 = Yes, 0 = No"),
        gr.Radio([0, 1], label="Sender Balance Empty After?", info="1 = Yes, 0 = No"),
        gr.Radio([0, 1], label="Recipient Balance Empty Before?", info="1 = Yes, 0 = No"),
        gr.Radio([0, 1], label="Recipient Balance Empty After?", info="1 = Yes, 0 = No"),
    ],
    outputs="text",
    title="💸 Fraud Detection App (Random Forest)",
    description="Enter transaction data to predict the likelihood of fraud using a trained ML model.",
)

In [18]:
demo.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
